In [2]:
#Create one hot sparse matrix for app installed and app label

In [3]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

app_events = pd.read_csv('./input/app_events.csv', \
                         usecols=['event_id', 'app_id', 'is_active'], \
                         dtype={'is_active':bool})
app_labels = pd.read_csv('./input/app_labels.csv', index_col='app_id')
events = pd.read_csv('./input/events.csv', index_col='event_id')
#gender_age_test = pd.read_csv('./input/gender_age_test.csv', index_col='device_id')
gender_age_train = pd.read_csv('./input/gender_age_train.csv', index_col='device_id')
#label_categories = pd.read_csv('./input/label_categories.csv', index_col='label_id')

In [4]:
from sklearn.preprocessing import LabelEncoder

app_encoder = LabelEncoder().fit(app_events.app_id)
app_events['app_id_encode'] = app_encoder.transform(app_events.app_id)
print("Number of apps", len(app_encoder.classes_))

Number of apps 19237


In [5]:
gender_age_train['trainrow'] = np.arange(len(gender_age_train.index))

In [6]:
gender_age_train.head()

,gender,age,group,trainrow
device_id,,,,
-8076087639492063270,M,35,M32-38,0
-2897161552818060146,M,35,M32-38,1
-8260683887967679142,M,35,M32-38,2
-4938849341048082022,M,30,M29-31,3
245133531816851882,M,30,M29-31,4


In [7]:
result = app_events\
        .merge(events[['device_id']], \
                 how='left', \
                 left_on="event_id", \
                 right_index=True) \
        .merge(gender_age_train[['trainrow']],\
               how="right",\
               left_on='device_id',\
               right_index=True) \
        .groupby(['trainrow','app_id_encode'])['app_id_encode'].agg(['size']) \
        .reset_index()

In [8]:
result.app_id_encode = result.app_id_encode.astype(np.int64)

In [9]:
result.head()

,trainrow,app_id_encode,size
0,2,135,1
1,2,267,1
2,2,317,1
3,2,526,1
4,2,974,1


In [10]:
from scipy.sparse import csr_matrix, hstack
apps_train = csr_matrix((
        np.ones(result.shape[0]), 
        (result.trainrow, result.app_id_encode)), 
        shape=(gender_age_train.shape[0],len(app_encoder.classes_)))

In [11]:
np.savez('intermediates/apps_train', \
         data = apps_train.data, \
         indices = apps_train.indices, \
         indptr = apps_train.indptr,)

In [12]:
### Process app labels here

In [13]:
app_labels.head()

,label_id
app_id,
7324884708820027918,251
-4494216993218550286,251
6058196446775239644,406
6058196446775239644,407
8694625920731541625,406


In [14]:
# Some of the app ids in app_labels are not in app_event
app_labels = app_labels.loc[app_labels.index.isin(app_events.app_id.unique())]
label_id_encoder = LabelEncoder().fit(app_labels.label_id)
app_labels['label_id_encode'] = label_id_encoder.transform(app_labels.label_id)
app_labels['app_id_encode'] = app_encoder.transform(app_labels.index)
print("Numbers of label id", len(label_id_encoder.classes_))

Numbers of label id 492


In [15]:
app_labels.head()

,label_id,label_id_encode,app_id_encode
app_id,,,
7324884708820027918,251,207,17355
-4494216993218550286,251,207,4618
6058196446775239644,406,247,15548
6058196446775239644,407,248,15548
8694625920731541625,406,247,18689


In [16]:
app_label_result = result.merge(app_labels[['label_id_encode','app_id_encode']], \
                                how='left',\
                                left_on='app_id_encode',\
                                right_on='app_id_encode')\
                        .groupby(['trainrow','label_id_encode'])['label_id_encode'].agg(['size'])\
                        .reset_index()
    

In [17]:
app_label_result.head()

,trainrow,label_id_encode,size
0,2,10,1
1,2,63,1
2,2,100,3
3,2,101,2
4,2,102,1


In [18]:
apps_label_train = csr_matrix((
        np.ones(app_label_result.shape[0]), 
        (app_label_result.trainrow, app_label_result.label_id_encode)), 
        shape=(gender_age_train.shape[0],len(label_id_encoder.classes_)))

In [19]:
np.savez('intermediates/label_train', \
         data = apps_label_train.data, \
         indices = apps_label_train.indices, \
         indptr = apps_label_train.indptr,)